<a href="https://colab.research.google.com/github/yashrajkakkad/ml-spotify/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml-spotify/data_merged.csv')
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence,year,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,minor,major,acousticness_ar,danceability_ar,duration_ms_ar,energy_ar,instrumentalness_ar,liveness_ar,loudness_ar,speechiness_ar,tempo_ar,valence_ar,popularity_ar,count,key_0_ar,key_1_ar,key_2_ar,key_3_ar,key_4_ar,key_5_ar,key_6_ar,key_7_ar,key_8_ar,key_9_ar,key_10_ar,key_11_ar,minor_ar,major_ar,acousticness_yr,danceability_yr,duration_ms_yr,energy_yr,instrumentalness_yr,liveness_yr,loudness_yr,speechiness_yr,tempo_yr,valence_yr,popularity_yr,mode,key_0_yr,key_1_yr,key_2_yr,key_3_yr,key_4_yr,key_5_yr,key_6_yr
0,0.0131,0.2560,182347,0.895,0,0.000106,0.0821,-4.860,29,0.0707,191.307,0.5550,1978,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.145175,0.523720,221014.800000,0.726840,0.023547,0.174298,-7.711613,0.045116,132.180853,0.701493,36.833333,150.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.341064,0.538997,247988.145057,0.552657,0.103383,0.217524,-11.095111,0.064961,120.209319,0.583424,35.272231,1,0,0,1,0,0,0,0
1,0.9800,0.2770,206972,0.145,0,0.879000,0.1110,-19.898,0,0.0845,75.644,0.4940,1953,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.962233,0.464711,199313.805031,0.217114,0.549370,0.166036,-14.844384,0.071003,102.967642,0.539736,2.974843,159.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.891747,0.440834,216552.259500,0.267418,0.314625,0.221442,-15.414304,0.092591,110.008113,0.432251,3.672500,1,1,0,0,0,0,0,0
2,0.7950,0.6850,314667,0.483,0,0.878000,0.1130,-10.202,1,0.0337,97.694,0.8540,1954,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.866200,0.630400,291778.600000,0.567800,0.895600,0.114200,-10.003400,0.040800,117.398600,0.837800,6.800000,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.867413,0.460371,225951.758500,0.263561,0.291320,0.216008,-15.342991,0.103243,108.561912,0.447291,7.707000,1,0,0,0,1,0,0,0
3,0.6560,0.7880,179747,0.808,0,0.000000,0.1540,-6.590,0,0.0395,113.046,0.9690,1954,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.721667,0.739333,175231.000000,0.745000,0.000001,0.077933,-4.730333,0.138767,127.433000,0.929667,2.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.867413,0.460371,225951.758500,0.263561,0.291320,0.216008,-15.342991,0.103243,108.561912,0.447291,7.707000,1,0,0,0,1,0,0,0
4,0.3020,0.0753,498560,0.150,0,0.884000,0.1210,-16.705,0,0.0371,76.558,0.0351,1951,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.817780,0.196522,521790.596429,0.309415,0.665675,0.246221,-15.794201,0.041948,94.257485,0.111619,1.666667,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.900334,0.461423,212773.310500,0.257270,0.303290,0.219540,-15.724956,0.107351,109.569882,0.443625,3.419500,1,1,0,0,0,0,0,0


In [ ]:
df.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence', 'year', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4',
       'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11',
       'minor', 'major', 'acousticness_ar', 'danceability_ar',
       'duration_ms_ar', 'energy_ar', 'instrumentalness_ar', 'liveness_ar',
       'loudness_ar', 'speechiness_ar', 'tempo_ar', 'valence_ar',
       'popularity_ar', 'count', 'key_0_ar', 'key_1_ar', 'key_2_ar',
       'key_3_ar', 'key_4_ar', 'key_5_ar', 'key_6_ar', 'key_7_ar', 'key_8_ar',
       'key_9_ar', 'key_10_ar', 'key_11_ar', 'minor_ar', 'major_ar',
       'acousticness_yr', 'danceability_yr', 'duration_ms_yr', 'energy_yr',
       'instrumentalness_yr', 'liveness_yr', 'loudness_yr', 'speechiness_yr',
       'tempo_yr', 'valence_yr', 'popularity_yr', 'mode', 'key_0_yr',
       'key_1_yr', 'key_2_yr', 'key_3_yr', 'ke

In [ ]:
mean_popularity = 42
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df.popularity.value_counts()  

0    127131
1     45099
Name: popularity, dtype: int64

In [ ]:
y = df["popularity"].values
X = df.drop(["popularity", "mode", "loudness", "loudness_ar", "loudness_yr"],axis=1)

In [ ]:
X.shape, y.shape

((172230, 67), (172230,))

In [ ]:
%%time

from sklearn.feature_selection import SelectPercentile, chi2
X = SelectPercentile(chi2, percentile=10).fit_transform(X, y)

input_scaler = StandardScaler()
X_normalized = input_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.20, random_state=42, stratify=y
)

CPU times: user 320 ms, sys: 119 ms, total: 439 ms
Wall time: 319 ms


In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from joblib import dump


svm = SVC(kernel='linear',
          C=1000,
          random_state=42)
svm.fit(X_train[:10000], y_train[:10000])
dump(svm, "mlspotify_SVM")

print("Train accuracy of svm algo:", svm.score(X_train[:10000], y_train[:10000]))
print("Test accuracy of svm algo:", svm.score(X_test, y_test))

# cv_scores = cross_val_score(svm, X_train[:20000], y_train[:20000])
# print("Cross Validation scores - ", cv_scores)

# y_train_pred = cross_val_predict(svm, X_train[:20000], y_train[:20000])

# confusion_matrix = confusion_matrix(y_train[:20000], y_train_pred)
# print("Confusion matrix\n", confusion_matrix)

# p_score = precision_score(y_train[:20000], y_train_pred)
# rc_score = recall_score(y_train[:20000], y_train_pred)
# f1_score = f1_score(y_train[:20000], y_train_pred)
# p_score, rc_score, f1_score
# print("Recall score = ", rc_score)
# print("Precision score = ", p_score)
# print("F1 score = ", f1_score)

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

Train accuracy of svm algo: 0.8749
Test accuracy of svm algo: 0.8761539801428323
[[23984  1442]
 [ 2824  6196]]
Classification report

              precision    recall  f1-score   support

           0       0.89      0.94      0.92     25426
           1       0.81      0.69      0.74      9020

    accuracy                           0.88     34446
   macro avg       0.85      0.82      0.83     34446
weighted avg       0.87      0.88      0.87     34446

CPU times: user 1min 19s, sys: 97.5 ms, total: 1min 19s
Wall time: 1min 19s


In [ ]:
from joblib import load
clf = load("mlspotify_SVM")
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

[[23984  1442]
 [ 2824  6196]]
Classification report

              precision    recall  f1-score   support

           0       0.89      0.94      0.92     25426
           1       0.81      0.69      0.74      9020

    accuracy                           0.88     34446
   macro avg       0.85      0.82      0.83     34446
weighted avg       0.87      0.88      0.87     34446

